In [1]:
print ('running part 2')
print ()
!python part2.py

print ()

print ('running part 3')
print ()
!python part3.py

print ()

print ('running part 4')
print ()
!python part4.py

print ()

print ('running part 5')
print ()
!python part5.py



running part 5

============================ EN ============================
time to get predictions for EN :  2.3038339614868164

EN
Entities in gold data : 802
Entities in prediction: 872

entities
 - Correct   : 500
 - Precision : 0.5734
 - Recall    : 0.6234
 - F score   : 0.5974

entity types
 - Correct   : 406
 - Precision : 0.4656
 - Recall    : 0.5062
 - F score   : 0.4851

k values:
[0.0, 0.09174311926605505, 0.1834862385321101, 0.27522935779816515, 0.3669724770642202, 0.45871559633027525, 0.5504587155963303, 0.6422018348623854, 0.7339449541284404, 0.8256880733944955, 0.9174311926605505, 1.0091743119266057, 1.1009174311926606, 1.1926605504587156, 1.2844036697247707, 1.3761467889908259, 1.4678899082568808, 1.5596330275229358, 1.651376146788991, 1.743119266055046, 1.834862385321101, 1.926605504587156, 2.0183486238532113, 2.1100917431192663, 2.2018348623853212, 2.293577981651376, 2.385321100917431, 2.4770642201834865, 2.5688073394495414, 2.6605504587155964, 2.7522935779816518, 2.